<a href="https://colab.research.google.com/github/IsraelMelMon/EfficientNet_Tensorflow2_Demo/blob/main/Efficient_TensorFlow2_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


In this notebook, we implement [The TensorFlow 2 Object Detection Library](https://blog.tensorflow.org/2020/07/tensorflow-2-meets-object-detection-api.html) for training on your own dataset.

You must have used an annotation tool to obtain the corresponding bounding box .xml files (each .xml having the corresponding images' name) using LabelImg or Maximo


# Install TensorFlow2 Object Detection Dependencies

In [1]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [2]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1613294 sha256=1e7162ef117249f4a1e1d74ac466e5bbef4021f0b395c18e9ce754443343ff1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lfvaa7zl/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

Test to see if the libraries were properly built.

In [4]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


2021-01-21 10:44:04.512395: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-01-21 10:44:07.471210: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-21 10:44:07.472306: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-21 10:44:07.491133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-21 10:44:07.491890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB de

In [5]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  #img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = path#Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

# Prepare Tensorflow 2 Object Detection Training Data

We need to create your TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

**You will need your custom dataset together with xml files (generated with Imagelbl or Maximo) in /content/train and /content/test as well as a label_map containing the classes**


**There is already a small dataset in this Colab but you ideally should train with thousands of images for best results **

We need one TFRecord file for our training data, and one TFRecord file for our test data, we'll upload two separate datasets and generate one set of TFRecords for each. We do this next:




## We import our test and train images from git with respective xml bounding boxes (from Maximo's Visual Inspection annotation tool)

In [6]:
#!cd /content/

In [7]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "train" in pathlib.Path.cwd().parts:
  while "train" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('train').exists():
  !git clone https://github.com/IsraelMelMon/EfficientNet_Tensorflow2_Demo/


fatal: destination path 'EfficientNet_Tensorflow2_Demo' already exists and is not an empty directory.


##If you used ImageLbl:

In [8]:
#import os
#os.environ['NET_DIR'] = "/content/EfficientNet_Tensorflow2_Demo"
#!python $NET_DIR/generate_tfrecord.py -x test/ -l  $NET_DIR/label_map.pbtxt -o $NET_DIR/test/test.tfrecord -c $NET_DIR/test/test_output.csv
#!python $NET_DIR/generate_tfrecord.py -x train/ -l $NET_DIR/label_map.pbtxt -o $NET_DIR/train/train.tfrecord -c $NET_DIR/train/train_output.csv

##If you used Maximo to obtain the xml files (our demo case):

In [26]:
import os
os.environ['NET_DIR'] = "/content/EfficientNet_Tensorflow2_Demo"
!python /content/EfficientNet_Tensorflow2_Demo/maximo_generate_tfrecord.py -x /content/EfficientNet_Tensorflow2_Demo/test/ -l  /content/EfficientNet_Tensorflow2_Demo/label_map.pbtxt -o/content/EfficientNet_Tensorflow2_Demo/test/test.tfrecord -c /content/EfficientNet_Tensorflow2_Demo/test/test_output.csv
!python /content/EfficientNet_Tensorflow2_Demo/maximo_generate_tfrecord.py -x /content/EfficientNet_Tensorflow2_Demo/train/ -l /content/EfficientNet_Tensorflow2_Demo/label_map.pbtxt -o /content/EfficientNet_Tensorflow2_Demo/train/train.tfrecord -c /content/EfficientNet_Tensorflow2_Demo/train/train_output.csv

checked
xml_stuff
path /content/EfficientNet_Tensorflow2_Demo/test/
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.xml
checking
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.jpg

29 1280
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.xml
checking
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.jpg

2 104
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.xml
checking
/content/EfficientNet_Tensorflow2_Demo/test/3b45425d-9656-46bd-aa60-2035506c9711.jpg

58 55
/content/EfficientNet_Tensorflow2_Demo/test/31352d0e-9ea5-41b5-9106-81da97253dc1.xml
checking
/content/EfficientNet_Tensorflow2_Demo/test/31352d0e-9ea5-41b5-9106-81da97253dc1.jpg

62 735
/content/EfficientNet_Tensorflow2_Demo/test/31352d0e-9ea5-41b5-9106-81da97253dc1.xml
checking
/content/EfficientNet_Tensorflow2_Demo/test/31352d0e-9ea5-41b5-9106-81da97253dc1.jpg

6 154

In [41]:
# NOTE: Update these TFRecord names
test_record_fname = '/content/EfficientNet_Tensorflow2_Demo/test/test.tfrecord'
train_record_fname = '/content/EfficientNet_Tensorflow2_Demo/train/train.tfrecord'
label_map_pbtxt_fname = '/content/EfficientNet_Tensorflow2_Demo/label_map.pbtxt'

Note: If saving as format tfrecord or pbtxt cause errors, then save them as text files and then in Google Drive change their names.

# Configure Custom TensorFlow2 Object Detection Training Configuration




> In this section you can specify any model in the [TF2 OD model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and set up your training configuration.



In [42]:
%cd /content

/content


In [43]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 8
    }
}


##We choose our neural network architecture here

In [44]:

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'efficientdet-d0'


##We choose our number of epochs here

In [45]:

num_steps = 1000 #int(float(100))            #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 10             #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [46]:
#download pretrained weights
%mkdir /content/models/research/deploy/
%cd /content/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

mkdir: cannot create directory ‘/content/models/research/deploy/’: File exists
/content/models/research/deploy
--2021-01-21 11:01:17--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.189.128, 2404:6800:4008:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.189.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz.6’

efficientdet_d0_coc 100%[===================>]  29.31M   194MB/s    in 0.2s    

2021-01-21 11:01:18 (194 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz.6’ saved [30736482/30736482]



In [47]:
#download base training configuration file
%cd /content/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/models/research/deploy
--2021-01-21 11:01:18--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.6’

ssd_efficientdet_d0 100%[===================>]   4.52K  --.-KB/s    in 0s      

2021-01-21 11:01:18 (57.2 MB/s) - ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.6’ saved [4630/4630]



We use the pretrained model weight files.


In [48]:
#prepare
pipeline_fname = '/content/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'
#fine_tune_checkpoint ="/content/training/ckpt-3.data-00000-of-00001"
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=2, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)


In [49]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)



/content/models/research/deploy
writing custom configuration file


## The architecture layers are shown in the **pipeline_config.config** file

In [50]:
%cat /content/models/research/deploy/pipeline_file.config

 # SSD with EfficientNet-b0 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d0).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b0 checkpoint.
#
# Train on TPU-8

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    add_background_class: false
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      m

## The trained model will be saved in the folder: **/content/fine_tuned_model/**

## IMPORTANT: If this is the first time training then do: 

In [51]:
#pipeline_file = '/content/fine_tuned_model/pipeline.config'
pipeline_file="/content/models/research/deploy/pipeline_file.config"
model_dir = '/content/training/'

## If this is a fine-tuning of a model then:

In [52]:
#pipeline_file = '/content/fine_tuned_model/pipeline.config'

# Train Custom TF2 Object Detector

* pipeline_file: defined above in writing custom training configuration
* model_dir: the location tensorboard logs and saved model checkpoints will save to
* num_train_steps: how long to train for
* num_eval_steps: perform eval on validation set after this many steps







In [53]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#trying to get rid of deprecation warnings help

##Training epochs should start 3-4 minutes after running the next cell.

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2021-01-21 11:01:22.855628: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-21 11:01:25.851675: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-21 11:01:25.852932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-21 11:01:25.877983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-21 11:01:25.878756: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-01-21 11:01:25.878822: I tensorflow/stream_executor/platform/default/dso_lo

## Observe the loss graphs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'
%reload_ext tensorboard %tensorboard --logdir '/content/training/train'


## Exporting a Trained Inference Graph
Still to come for TF2 models, we will be updating this Colab notebook accordingly as the functionality is added. 

In [ ]:
#see where our model saved weights
%ls '/content/training/'

checkpoint                  ckpt-4.index
ckpt-1.data-00000-of-00001  ckpt-5.data-00000-of-00001
ckpt-1.index                ckpt-5.index
ckpt-2.data-00000-of-00001  ckpt-6.data-00000-of-00001
ckpt-2.index                ckpt-6.index
ckpt-3.data-00000-of-00001  ckpt-7.data-00000-of-00001
ckpt-3.index                ckpt-7.index
ckpt-4.data-00000-of-00001  train/


In [ ]:
from imutils.video import FPS
import numpy as np
import argparse
import cv2

In [ ]:
#run conversion script
import re
import numpy as np

output_directory = '/content/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/content/training/'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [ ]:
%ls '/content/fine_tuned_model/saved_model/'

assets/  saved_model.pb  variables/


Remember to save the **saved_model.pb**, all the files in the **checkpoint file**, and the **variable file**.

# Run Inference on Test Video with Custom TensorFlow2 Object Detector

## You must load your video onto **/content/video.mov**

In [ ]:
#downloading test images from Roboflow
#export dataset above with format COCO JSON
#or import your test images via other means. 
%mkdir /content/test/
%cd /content/test/

In [ ]:

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
%ls '/content/training/'

checkpoint                  ckpt-4.index
ckpt-1.data-00000-of-00001  ckpt-5.data-00000-of-00001
ckpt-1.index                ckpt-5.index
ckpt-2.data-00000-of-00001  ckpt-6.data-00000-of-00001
ckpt-2.index                ckpt-6.index
ckpt-3.data-00000-of-00001  ckpt-7.data-00000-of-00001
ckpt-3.index                ckpt-7.index
ckpt-4.data-00000-of-00001  train/


These are the weight files. One can continue training automatically if one specifies the weight file as shown below.

In [ ]:
#recover our saved model
pipeline_config = pipeline_file
#generally you want to put the last ckpt from training in here
model_dir = '/content/fine_tuned_model/checkpoint/ckpt-0'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/fine_tuned_model/checkpoint/ckpt-0'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

Nex, we hard-code our video input name. One can also run one image at a time uncommenting the glob part.

In [24]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random
import cv2
from imutils.video import FPS

#input video for object detection inference
vid = cv2.VideoCapture('/content/video.mov')
ret, im = vid.read()
imshape = im.shape
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

#print("same shapes?",im.shape, (im.shape[1],im.shape[0]))


#output video name
videoOut = cv2.VideoWriter('/content/detectVideo_Output2.avi',fourcc, 20.0, (im.shape[1],im.shape[0]))

print("[INFO] loading model...")
print("[INFO] starting video play...")
fps = FPS().start()

counter = 1
while True:

  ret, frame = vid.read()

  if ret:
        

    #TEST_IMAGE_PATHS = glob.glob('/content/test/*.jpg')


    #[file for file in glob.glob('/content/test/*.jpg')]


    #image_path = random.choice(TEST_IMAGE_PATHS)
    (im_width, im_height) = (frame.shape[1],frame.shape[0])
    #print(im_width, im_height)

     
    image_np = np.array(frame).reshape((im_height, im_width, 3)).astype(np.uint8)#load_image_into_numpy_array(image_path)
    #print(image_np)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int)-1,
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5,
          agnostic_mode=False,
    )

    #plt.figure(figsize=(12,16))
    #plt.imshow(image_np_with_detections)
    #plt.show()
    #cv2.imwrite("check{0}.jpg".format(counter), cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR))
    videoOut.write(image_np_with_detections)
    fps.update()
    counter = counter + 1
  else:
    break
fps.stop()

print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

videoOut.release()


AttributeError: ignored

In [ ]:
pwd

'/content/models/research/deploy'